<a href="https://colab.research.google.com/github/StacyChebet/Dyslexicon/blob/master/dyslexicon2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dyslexicon CNN Model**

Dyslexicon is a CNN model that predicts the likelihood that a child is suffering from Dyslexia.

In [1]:
%cd /content/drive/MyDrive/Project
%ls

/content/drive/MyDrive/Project
'Dataset Dyslexia_Password WanAsy321.rar'   Gambo/
 dyslexiaHandwriting/                       Validation/
 dyslexiaHandwriting.zip


In [2]:
#Importing modules
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import os
import json
import PIL
import PIL.Image
import tensorflow_datasets as tfds
import pathlib
import cv2
from keras.applications import ResNet152
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [3]:
!pip install unrar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install tensorflow==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
! unzip /content/drive/MyDrive/Project/dyslexiaHandwriting.zip


Archive:  /content/drive/MyDrive/Project/dyslexiaHandwriting.zip
replace Dataset Dyslexia_Password WanAsy321.rar? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
! unrar x -Y /content/drive/MyDrive/dyslexiaHandwriting.rar

Streaming output truncated to the last 5000 lines.
Extracting  Gambo/Test/Normal/Normal413 (3).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal413 (4).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal413 (5).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal413 (6).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal413.png                               14%  OK 
Extracting  Gambo/Test/Normal/Normal414 (2).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal414 (3).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal414 (4).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal414 (5).png                           14%  OK 
Extracting  Gambo/Test/Normal/Normal414 (6).png                           14%  OK 
Extracting  Gambo/Test/Normal

In [ ]:
trainPath = 'Gambo/Train'
testPath = 'Gambo/Test'
validationPath = '/content/drive/MyDrive/Project/Validation'

In [ ]:
train_dir = pathlib.Path(trainPath)
test_dir = pathlib.Path(validationPath)

In [ ]:
print(str(len(list(train_dir.glob('*/*.png'))) )+ ' Training Images')
print(str(len(list(test_dir.glob('*/*.png')))) + ' Testing Images')

In [ ]:
batch_size = 32
image_width = 224
image_height = 224

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split = 0.1,
    subset ='training',
    seed = 123,
    image_size = (image_height, image_width),
    batch_size = batch_size
)

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split = 0.1,
    subset ='validation',
    seed = 123,
    image_size = (image_height, image_width),
    batch_size = batch_size
)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3,3, i+1)
    plt.imshow(images[i].numpy().astype('uint8'))
    plt.title(class_names[labels[i]])
    plt.axis('off')


In [ ]:
#The image batch is a tensor of the shape
for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)

In [ ]:
num_classes = len(class_names)
print(num_classes)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
val_dataset = validation_dataset.cache().prefetch(buffer_size = AUTOTUNE)

**Reshaping the Images**

Reshaping the images is a necessary step since the model takes images of specific dimensions (224 x 224) and the images come in dimensions of (300 x 20).

In [ ]:
resizeAndRescale = tf.keras.Sequential([
    layers.Resizing(image_height, image_width),
    layers.Rescaling(1./255)
])

**Data Augmentation**<br>
This is the process of adding new data artificially derived from existing training data with techniques like resizing, rotating, flipping, cropping, padding and more, to address issues like overfitting and make the model more robust with better performance.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(image_height,
                                  image_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)
 

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
num_classes = len(class_names)
resnet_model = Sequential([
    resizeAndRescale,
  data_augmentation
])

pretrained_model= tf.keras.applications.ResNet152(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes= num_classes,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)

**Early Stopping**<br>
Early stopping is a form of regularization used to prevent overfitting. 

In [ ]:

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(5, activation='softmax'))

In [ ]:
resnet_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

In [ ]:
epochs = 20
history = resnet_model.fit(
    train_dataset, 
    validation_data=validation_dataset, 
    epochs=epochs, 
    callbacks = [early_stop],
    shuffle=True)

# **Evaluation of the Model**

**Accuracy**

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

**Loss**

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

**Predictions**

***Normal Predictions***

In [ ]:
test_image = ''
img = tf.keras.utils.load_img(
    test_image, target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = resnet_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

***Dyslexic Predictions***

In [ ]:
test_image = ''
img = tf.keras.utils.load_img(
    test_image, target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = resnet_model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)